In [ ]:
%pip install transformers==4.45.0 datasets evaluate seqeval

In [ ]:
from huggingface_hub import notebook_login

notebook_login()

In [ ]:
from datasets import Dataset, DatasetDict
import pandas as pd

datasetName = "play_audio"
df = pd.read_json(f"../../train/data/{datasetName}.json")
dataset = Dataset.from_pandas(df)
#dataset = dataset.class_encode_column("ner_tags")
trainTest = dataset.train_test_split(0.2)
testVal = trainTest["test"].train_test_split(0.3)
dataset = DatasetDict({
    "train": trainTest["train"],
    "test": testVal["train"],
    "validation": testVal["test"]
})
dataset

In [ ]:
dataset["train"][0]

In [ ]:
label_list = [
    "O",
    "B-SongName",
    "I-SongName",
    "B-ArtistName",
    "I-ArtistName"
]

#print(label_list)

In [ ]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("distilbert/distilbert-base-uncased")

In [ ]:
example = dataset["train"][0]

tokenized_input = tokenizer(example["tokens"], is_split_into_words=True)

tokens = tokenizer.convert_ids_to_tokens(tokenized_input["input_ids"])

tokens

In [ ]:
def tokenize_and_align_labels(examples):
    tokenized_inputs = tokenizer(examples["tokens"], truncation=True, is_split_into_words=True)

    labels = []
    for i, label in enumerate(examples[f"ner_tags"]):
        word_ids = tokenized_inputs.word_ids(batch_index=i)  # Map tokens to their respective word.
        previous_word_idx = None
        label_ids = []
        for word_idx in word_ids:  # Set the special tokens to -100.
            if word_idx is None:
                label_ids.append(-100)
            elif word_idx != previous_word_idx:  # Only label the first token of a given word.
                label_ids.append(label[word_idx])
            else:
                label_ids.append(-100)
            previous_word_idx = word_idx
        labels.append(label_ids)

    tokenized_inputs["labels"] = labels
    return tokenized_inputs

In [ ]:
tokenized_wnut = dataset.map(tokenize_and_align_labels, batched=True)

In [ ]:
from transformers import DataCollatorForTokenClassification

data_collator = DataCollatorForTokenClassification(tokenizer=tokenizer, return_tensors="tf")

In [ ]:
import evaluate

seqeval = evaluate.load("seqeval")

In [ ]:
import numpy as np

labels = [label_list[i] for i in example[f"ner_tags"]]


def compute_metrics(p):

    predictions, labels = p

    predictions = np.argmax(predictions, axis=2)

    true_predictions = [

        [label_list[p] for (p, l) in zip(prediction, label) if l != -100]

        for prediction, label in zip(predictions, labels)

    ]

    true_labels = [

        [label_list[l] for (p, l) in zip(prediction, label) if l != -100]

        for prediction, label in zip(predictions, labels)

    ]

    results = seqeval.compute(predictions=true_predictions, references=true_labels)

    return {

        "precision": results["overall_precision"],

        "recall": results["overall_recall"],

        "f1": results["overall_f1"],

        "accuracy": results["overall_accuracy"],

    }

In [ ]:
id2label = {x: label_list[x] for x in range(len(label_list))}

label2id = {v: k for k, v in id2label.items()}

In [ ]:
from transformers import create_optimizer

batch_size = 16
num_train_epochs = 3
num_train_steps = (len(tokenized_wnut["train"]) // batch_size) * num_train_epochs
optimizer, lr_schedule = create_optimizer(
    init_lr=2e-5,
    num_train_steps=num_train_steps,
    weight_decay_rate=0.01,
    num_warmup_steps=0,
)

In [ ]:
from transformers import TFAutoModelForTokenClassification

model = TFAutoModelForTokenClassification.from_pretrained(
    "distilbert/distilbert-base-uncased", num_labels=len(label_list), id2label=id2label, label2id=label2id
)

In [ ]:
tf_train_set = model.prepare_tf_dataset(
    tokenized_wnut["train"],
    shuffle=True,
    batch_size=16,
    collate_fn=data_collator,
)

tf_validation_set = model.prepare_tf_dataset(
    tokenized_wnut["validation"],
    shuffle=False,
    batch_size=16,
    collate_fn=data_collator,
)

In [ ]:
import tensorflow as tf

model.compile(optimizer=optimizer, run_eagerly=True)

In [ ]:
from transformers.keras_callbacks import KerasMetricCallback

metric_callback = KerasMetricCallback(metric_fn=compute_metrics, eval_dataset=tf_validation_set)

In [ ]:
from transformers.keras_callbacks import PushToHubCallback

push_to_hub_callback = PushToHubCallback(
    output_dir="Project_Nigel_Slot_Filling",
    tokenizer=tokenizer
)

In [ ]:
callbacks = [metric_callback, push_to_hub_callback]

In [ ]:
model.fit(x=tf_train_set, validation_data=tf_validation_set, epochs=1, callbacks=callbacks)

In [ ]:
import os

modelName = "0.1"

if not os.path.exists("../models"):
    os.mkdir("../models")
    
modelDir = f"../models/{modelName}/"
if not os.path.exists(modelDir):
    os.mkdir(modelDir)
model.save_pretrained(f"{modelDir}/")

In [ ]:
text = "i want you to play too cool to be careless by big steve and the rocking chair"
inputs = tokenizer(text, return_tensors="tf")
logits = model(**inputs).logits
predicted_token_class_ids = tf.math.argmax(logits, axis=-1)

predicted_token_class = [model.config.id2label[t] for t in predicted_token_class_ids[0].numpy().tolist()]

predicted_token_class

In [ ]:
result = classifier()
print(result)
combined_entities = []
current_entity = None
current_label = None

for item in result:
    entity = item['entity']
    word = item['word']

    if entity.startswith('B-'):
        # Start of a new entity, so save the previous one
        if current_entity:
            combined_entities.append(f"{current_label}: {current_entity}")
        current_entity = word
        current_label = entity[2:]  # Remove 'B-' prefix
    elif entity.startswith('I-') and entity[2:] == current_label:
        # Continuation of the current entity
        current_entity += f" {word}"
    else:
        # Non-B/I entity, just append directly
        if current_entity:
            combined_entities.append(f"{current_label}: {current_entity}")
        current_entity = None
        current_label = None
        combined_entities.append(f"{entity}: {word}")

# Append the last entity if it exists
if current_entity:
    combined_entities.append(f"{current_label}: {current_entity}")

# Display the combined entities
for entity in combined_entities:
    print(entity)